In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('/content/dynamic_pricing.csv')

In [3]:
df.shape

(1000, 10)

In [4]:
df.isnull().sum()

,0
Number_of_Riders,0
Number_of_Drivers,0
Location_Category,0
Customer_Loyalty_Status,0
Number_of_Past_Rides,0
Average_Ratings,0
Time_of_Booking,0
Vehicle_Type,0
Expected_Ride_Duration,0
Historical_Cost_of_Ride,0


In [5]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Number_of_Riders         1000 non-null   int64  
 1   Number_of_Drivers        1000 non-null   int64  
 2   Location_Category        1000 non-null   object 
 3   Customer_Loyalty_Status  1000 non-null   object 
 4   Number_of_Past_Rides     1000 non-null   int64  
 5   Average_Ratings          1000 non-null   float64
 6   Time_of_Booking          1000 non-null   object 
 7   Vehicle_Type             1000 non-null   object 
 8   Expected_Ride_Duration   1000 non-null   int64  
 9   Historical_Cost_of_Ride  1000 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 78.3+ KB


In [6]:
df.describe()

,Number_of_Riders,Number_of_Drivers,Number_of_Past_Rides,Average_Ratings,Expected_Ride_Duration,Historical_Cost_of_Ride
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,60.372000,27.076000,50.031000,4.257220,99.58800,372.502623
std,23.701506,19.068346,29.313774,0.435781,49.16545,187.158756
min,20.000000,5.000000,0.000000,3.500000,10.00000,25.993449
25%,40.000000,11.000000,25.000000,3.870000,59.75000,221.365202
50%,60.000000,22.000000,51.000000,4.270000,102.00000,362.019426
75%,81.000000,38.000000,75.000000,4.632500,143.00000,510.497504
max,100.000000,89.000000,100.000000,5.000000,180.00000,836.116419


In [8]:
df.describe(include="object")

,Location_Category,Customer_Loyalty_Status,Time_of_Booking,Vehicle_Type
count,1000,1000,1000,1000
unique,3,3,4,2
top,Urban,Silver,Night,Premium
freq,346,367,276,522


In [9]:
df

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422
...,...,...,...,...,...,...,...,...,...,...
995,33,23,Urban,Gold,24,4.21,Morning,Premium,11,91.389526
996,84,29,Urban,Regular,92,4.55,Morning,Premium,94,424.155987
997,44,6,Suburban,Gold,80,4.13,Night,Premium,40,157.364830
998,53,27,Suburban,Regular,78,3.63,Night,Premium,58,279.095048


In [11]:
df['demand_supply_ratio'] = df['Number_of_Riders'] / (df['Number_of_Drivers'] + 1)

In [13]:
def dynamic_pricing(row):
    price = row['Historical_Cost_of_Ride']

    # Demand-Supply Impact
    if row['demand_supply_ratio'] > 1.5:
        price *= 1.25
    elif row['demand_supply_ratio'] < 0.8:
        price *= 0.9

    # Loyalty Discount
    if row['Customer_Loyalty_Status'] == 'Gold':
        price *= 0.9
    elif row['Customer_Loyalty_Status'] == 'Silver':
        price *= 0.95

    # Peak hour surcharge
    if row['Time_of_Booking'] in ['morning peak', 'evening peak']:
        price *= 1.15

    # Vehicle Type Premium
    if row['Vehicle_Type'] == 'SUV':
        price *= 1.2
    elif row['Vehicle_Type'] == 'Bike':
        price *= 0.8

    return round(price, 2)

df['Simulated_Price'] = df.apply(dynamic_pricing, axis=1)


In [15]:
df['Booking_Status'] = df['Number_of_Past_Rides'].apply(lambda x: 'Activated' if x > 0 else 'Onboarded')


In [16]:
df

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,demand_supply_ratio,Simulated_Price,Booking_Status
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273,1.956522,337.56,Activated
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753,1.450000,165.18,Activated
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469,1.312500,313.31,Onboarded
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232,3.068966,587.75,Activated
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422,3.391304,724.60,Activated
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,33,23,Urban,Gold,24,4.21,Morning,Premium,11,91.389526,1.375000,82.25,Activated
996,84,29,Urban,Regular,92,4.55,Morning,Premium,94,424.155987,2.800000,530.19,Activated
997,44,6,Suburban,Gold,80,4.13,Night,Premium,40,157.364830,6.285714,177.04,Activated
998,53,27,Suburban,Regular,78,3.63,Night,Premium,58,279.095048,1.892857,348.87,Activated


In [17]:
df.to_excel("Dynamic_Pricing_Simulator_Output.xlsx", index=False)
